In [1]:
import os
%pwd

'c:\\Users\\Akash R\\OneDrive\\Desktop\\Projectts\\End-to-End-Diamond-Price-Prediction\\NoteBook_Experiments'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\Akash R\\OneDrive\\Desktop\\Projectts\\End-to-End-Diamond-Price-Prediction'

In [18]:
#entity -> Custom return type for a function
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path

In [19]:
#Configuration Manager
from DiamondRegressor.constants import *
from DiamondRegressor.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url= config.source_url,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config


In [20]:
#Components
import os
from DiamondRegressor import logger
import urllib.request as request
import zipfile
from DiamondRegressor.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"Downloading {filename} with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)


In [21]:
try:
    config = ConfigurationManager()
    data_ingestion_config =  config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-31 19:08:00,202: INFO: common: YAML file: {path} loaded successfully]
[2023-12-31 19:08:00,208: INFO: common: YAML file: {path} loaded successfully]
[2023-12-31 19:08:00,214: INFO: common: created directory at: artifacts]
[2023-12-31 19:08:00,217: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-31 19:08:01,340: INFO: 1396340410: Downloading artifacts/data_ingestion/data.zip with the following info: 
Connection: close
Content-Length: 572448
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c6b91bbe88403a74f19bd0cdb8e807a575f20ddf5e22cb4f97c59eaca3560fbd"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A574:4B8D:AE98DB:DA1D69:65916E0B
Accept-Ranges: bytes
Date: Sun, 31 Dec 2023 13:38:01 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10245-MAA
X-Cache: HIT
X-Cache-Hit